In [1]:
import random
import numpy as np
import openai
import pandas as pd
import os
import sys
import time
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import openai
# Add the path to the constants file to the system path
sys.path.append('../../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *
from CF_utils import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, '../data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'ml-1m/merged_data.dat')
print(f'Data path: {data_path}')

# output

CF_output_path = os.path.join(DATA_DIR, 'ml-1m/output/CF_output_path_2_ratings_per_user.dat')
print(f'Data path: {data_path}')

CF_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/rerun_CF_output_path_2_ratings_per_user.dat')
print(f'Data path: {data_path}')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/notebook
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/notebook/../data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/notebook/../data/ml-1m/merged_data.dat
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/notebook/../data/ml-1m/merged_data.dat
Data path: /Users/tnathu-ai/VSC

In [3]:
data = pd.read_csv(data_path)
data.head(3)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama


In [4]:
# Create User-Item Interaction Matrix
interaction_matrix = pd.pivot_table(data, index='UserID', columns='MovieID', values='Rating').fillna(0)
csr_interaction_matrix = csr_matrix(interaction_matrix.values)

# Calculate Pearson Correlation Coefficient Matrix
pcc_matrix = pearson_correlation(csr_interaction_matrix)

pcc_matrix

array([[ 1.        ,  0.41666667, -0.33218192, ...,  0.        ,
         0.05685735, -0.04351941],
       [ 0.41666667,  1.        ,  0.23683386, ..., -0.5       ,
         0.57207755, -0.0271435 ],
       [-0.33218192,  0.23683386,  1.        , ...,  0.5       ,
         0.30927686, -0.39528471],
       ...,
       [ 0.        , -0.5       ,  0.5       , ...,  1.        ,
         0.27116307, -0.39712226],
       [ 0.05685735,  0.57207755,  0.30927686, ...,  0.27116307,
         1.        ,  0.24230884],
       [-0.04351941, -0.0271435 , -0.39528471, ..., -0.39712226,
         0.24230884,  1.        ]])

In [5]:
csr_interaction_matrix

<6040x3706 sparse matrix of type '<class 'numpy.float64'>'
	with 1000209 stored elements in Compressed Sparse Row format>

In [6]:
interaction_matrix

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:


@retry_decorator
def predict_rating_combined_ChatCompletion(combined_text, 
                                           model=GPT_MODEL_NAME, 
                                           temperature=TEMPERATURE, 
                                           approach="zero-shot", 
                                           rating_history=None, 
                                           similar_users_ratings=None, 
                                           seed=RANDOM_STATE, 
                                           system_content=AMAZON_CONTENT_SYSTEM):
    # Validation
    if approach == "few-shot" and rating_history is None:
        raise ValueError("Rating history is required for the few-shot approach.")
    if approach == "CF" and similar_users_ratings is None:
        raise ValueError("Similar users' ratings are required for the collaborative filtering approach.")
    if not system_content:
        raise ValueError("System content is required.")

    # Check and reduce length of combined_text
    combined_text = check_and_reduce_length(combined_text, MAX_TOKENS_CHAT_GPT // 3, TOKENIZER)
    prompt = f"How will user rate this {combined_text}? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text."

    # Construct the prompt based on the approach
    if approach == "few-shot":
        rating_history = check_and_reduce_length(rating_history, MAX_TOKENS_CHAT_GPT // 3, TOKENIZER)
        prompt += f"\n\nHere is user rating history:\n{rating_history}"

    elif approach == "CF":
        rating_history = check_and_reduce_length(rating_history, MAX_TOKENS_CHAT_GPT // 3, TOKENIZER)
        prompt += f"\n\nHere is user rating history:\n{rating_history}"
        similar_users_ratings = check_and_reduce_length(similar_users_ratings, MAX_TOKENS_CHAT_GPT // 3, TOKENIZER)
        prompt += f"\n\nHere are the rating history from users who are similar to this user:\n{similar_users_ratings}"

    # Adding end of the prompt
    prompt += "\n\nBased on the above information, please predict user's rating for the product: (1 being lowest and 5 being highest, The output should be like: (x stars, xx%), do not explain the reason.)"

    print(f"Constructed Prompt for {approach} approach:\n")
    print(f'The prompt:\n**********\n{prompt}\n**********\n')

    try:
        # Create the API call
        response = openai.ChatCompletion.create(
            model=model,
            temperature=temperature,
            max_tokens=MAX_TOKENS_CHAT_GPT,
            seed=seed,
            messages=[
                {"role": "system", "content": system_content},
                {"role": "user", "content": prompt}
            ]
        )
        # Extract the system fingerprint and print it
        system_fingerprint = response.get('system_fingerprint')
        print(f"\n\nSystem Fingerprint: {system_fingerprint}")
        # Extract and return the rating
        rating_text = response.choices[0].message['content'].strip()
        print(f'\nAPI call response: "{rating_text}"')
        extracted_rating = extract_numeric_rating(rating_text)
        print(f'Extracted rating: {extracted_rating}')
        return extracted_rating  # Ensure this is a float
    
    except APIError as api_err:
        print(f"API Error occurred: {api_err}")
        return None, str(api_err)
    except RateLimitError as rate_err:
        print(f"Rate Limit Error occurred: {rate_err}")
        return None, str(rate_err)
    except Exception as e:
        print(f"Unexpected Error: {e}")
        return None, str(e)


def predict_ratings_zero_shot_and_save(data,
                                       columns_for_prediction=['title'],
                                       user_column_name='reviewerID',
                                       title_column_name='title',
                                       asin_column_name='asin',
                                       rating_column_name='rating',
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path='zero_shot_predictions.csv',
                                       seed=RANDOM_STATE):
    """
    Predicts a single random rating per user using a zero-shot approach and saves the predictions to a CSV file.

    Parameters:
    - data (DataFrame): Dataset containing user ratings.
    - columns_for_prediction (list of str): Columns to use for prediction.
    - user_column_name (str): Column name for user IDs.
    - title_column_name (str): Column name for item titles.
    - asin_column_name (str): Column name for item IDs.
    - rating_column_name (str): Column name for actual ratings.
    - pause_every_n_users (int): Number of users to process before pausing.
    - sleep_time (int): Sleep time in seconds during pause.
    - save_path (str): Path to save the predictions CSV file.
    - seed (int): Seed for random number generation.

    Returns:
    - DataFrame: DataFrame containing prediction results.
    """

    results = []
    random.seed(seed)

    # Group data by user and filter users with at least 5 records
    grouped_data = data.groupby(user_column_name).filter(lambda x: len(x) >= 5)
    unique_users = grouped_data[user_column_name].unique()

    for i, user_id in enumerate(unique_users):
        user_data = grouped_data[grouped_data[user_column_name] == user_id]
        # Select a random record for each user
        random_row = user_data.sample(n=1, random_state=seed).iloc[0]

        # Generate combined text for prediction using specified columns
        combined_text = ' | '.join([f"{col}: {random_row[col]}" for col in columns_for_prediction])

        # Predict rating using zero-shot approach
        predicted_rating = predict_rating_combined_ChatCompletion(combined_text, approach="zero-shot")
        item_id = random_row[asin_column_name]
        actual_rating = random_row[rating_column_name]
        title = random_row[title_column_name]

        results.append([user_id, item_id, title, actual_rating, predicted_rating])

        # Print progress and pause if necessary
        if (i + 1) % pause_every_n_users == 0:
            print(f"Processed {i + 1} users. Pausing for {sleep_time} seconds...")
            time.sleep(sleep_time)

    # Save results to CSV
    results_df = pd.DataFrame(results, columns=['user_id', 'item_id', 'title', 'actual_rating', 'predicted_rating'])
    results_df.to_csv(save_path, index=False)
    print(f"Predictions saved to {save_path}")

    return results_df



In [8]:
import random

def predict_ratings_with_collaborative_filtering_and_save(data, pcc_matrix, 
                                                          user_column_name='reviewerID', 
                                                          movie_column_name='title', 
                                                          movie_id_column='asin',
                                                          rating_column_name='rating', 
                                                          num_ratings_per_user=1, 
                                                          num_similar_users=4,
                                                          num_main_user_ratings=1,
                                                          save_path='cf_predictions.csv', 
                                                          seed=RANDOM_STATE):
    results = []
    unique_users = data[user_column_name].unique()
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(unique_users)}

    random.seed(seed)

    for user_id in unique_users:
        user_idx = user_id_to_index[user_id]

        print(f"Processing user {user_id} (Index: {user_idx})")

        # Retrieve the main user's historical ratings
        main_user_data = data[data[user_column_name] == user_id]
        if len(main_user_data) > num_main_user_ratings:
            main_user_ratings = main_user_data.sample(n=num_main_user_ratings, random_state=seed)
        else:
            main_user_ratings = main_user_data

        main_user_ratings_str = '\n'.join([
            f"* Title: {row[movie_column_name]}, Rating: {row[rating_column_name]} stars"
            for _, row in main_user_ratings.iterrows()
        ])

        # Find the top similar users based on Pearson Correlation Coefficient
        similar_users_idx = np.argsort(-pcc_matrix[user_idx])[:num_similar_users + 1]
        similar_users_idx = similar_users_idx[similar_users_idx != user_idx][:num_similar_users]

        print(f"Top similar users for {user_id}: {[unique_users[idx] for idx in similar_users_idx]}")

        # Collect historical ratings from similar users
        similar_users_ratings = ""
        for idx in similar_users_idx:
            similar_user_id = unique_users[idx]
            similar_user_data = data[data[user_column_name] == similar_user_id]
            if len(similar_user_data) > num_ratings_per_user:
                historical_ratings = similar_user_data.sample(n=num_ratings_per_user, random_state=seed)
            else:
                historical_ratings = similar_user_data

            for _, row in historical_ratings.iterrows():
                rating_info = f"* Title: {row[movie_column_name]}, Rating: {row[rating_column_name]} stars"
                similar_users_ratings += rating_info + "\n"

        # Select a random movie from the user's data for prediction
        random_movie_row = main_user_data.sample(n=1, random_state=seed).iloc[0]
        random_movie_title = random_movie_row[movie_column_name]
        random_movie_id = random_movie_row[movie_id_column]
        actual_rating = random_movie_row[rating_column_name]

        # Construct prompt for API call
        combined_text = f"Title: {random_movie_title}"
        prompt = f"Main User Ratings:\n{main_user_ratings_str}\n\nSimilar Users' Ratings:\n{similar_users_ratings}\n\nPredict rating for '{combined_text}':"

        print(f"Generated prompt for user {user_id}:\n{prompt}")

        predicted_rating = predict_rating_combined_ChatCompletion(
            combined_text, 
            approach="CF", 
            similar_users_ratings=similar_users_ratings,
            rating_history=main_user_ratings_str
        )

        # Store prediction results
        results.append([user_id, random_movie_id, random_movie_title, actual_rating, predicted_rating])

        print(f"User {user_id}: Predicted rating for '{random_movie_title}' is {predicted_rating}.")

    results_df = pd.DataFrame(results, columns=['user_id', 'item_id', 'title', 'actual_rating', 'predicted_rating'])
    results_df.to_csv(save_path, index=False)
    print(f"Predictions saved to {save_path}")

    return results_df


In [9]:
%%time


cf_predictions = predict_ratings_with_collaborative_filtering_and_save(data, 
                                                                       pcc_matrix, 
                                                                       save_path=CF_output_path,
                                                                       user_column_name='UserID',
                                                                       movie_column_name='Title',
                                                                       movie_id_column='MovieID',
                                                                       rating_column_name='Rating', 
                                                                       num_ratings_per_user=1,
                                                                       num_main_user_ratings=4,
                                                                       num_similar_users=4, )


Processing user 1 (Index: 0)
Top similar users for 1: [4871, 74, 503, 2099]
Generated prompt for user 1:
Main User Ratings:
* Title: Pleasantville (1998), Rating: 3 stars
* Title: Antz (1998), Rating: 4 stars
* Title: Dead Poets Society (1989), Rating: 4 stars
* Title: Sixth Sense, The (1999), Rating: 4 stars

Similar Users' Ratings:
* Title: Back to the Future (1985), Rating: 4 stars
* Title: Badlands (1973), Rating: 3 stars
* Title: Laura (1944), Rating: 5 stars
* Title: Almost Famous (2000), Rating: 4 stars


Predict rating for 'Title: Pleasantville (1998)':
Constructed Prompt for CF approach:

The prompt:
**********
How will user rate this Title: Pleasantville (1998)? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text.

Here is user rating history:
* Title: Pleasantville (1998), Rating: 3 stars
* Title: Antz (1998), Rating: 4 stars
* Title: Dead Poets Society (1989), Rating: 4 stars
* Title: Sixth Sense, 

In [ ]:
import pandas as pd

# Read the data
data = pd.read_csv(CF_output_path)

# Display the original data types
print("Original Data Types:")
print(data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
data['is_rating_float'] = pd.to_numeric(data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = data[data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# rerun indices for non-float ratings
rerun_indices = non_float_ratings.index.tolist()
print(f"Rerun indices: {rerun_indices}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


Original Data Types:
user_id              int64
item_id              int64
title               object
actual_rating        int64
predicted_rating    object
dtype: object


Total number of rows with non-float ratings: 29
Rerun indices: [212, 290, 464, 469, 614, 691, 721, 854, 863, 895, 1123, 1696, 1713, 1815, 2042, 2214, 2435, 3008, 3163, 3269, 3382, 3508, 4095, 4096, 4112, 4113, 4353, 4720, 5436]
Rows with non-float ratings:


,user_id,item_id,title,actual_rating,predicted_rating,is_rating_float
212,749,2001,Lethal Weapon 2 (1989),2,"(None, 'The server is overloaded or not ready ...",False
290,1036,1203,12 Angry Men (1957),4,"(None, 'The server is overloaded or not ready ...",False
464,1701,2336,Elizabeth (1998),5,"(None, 'The server is overloaded or not ready ...",False


In [ ]:
def rerun_and_update_failed_predictions(original_data_path, pcc_matrix, 
                                        failed_indices_df, 
                                        data, 
                                        user_column_name='reviewerID', 
                                        movie_column_name='title', 
                                        movie_id_column='asin',
                                        rating_column_name='rating', 
                                        num_ratings_per_user=1, 
                                        num_similar_users=4,
                                        num_main_user_ratings=1,
                                        seed=RANDOM_STATE):
    """
    Reruns predictions for rows with failed indices and updates the original DataFrame.
    """
    original_data = pd.read_csv(original_data_path)
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(data[user_column_name].unique())}

    random.seed(seed)

    for _, row in failed_indices_df.iterrows():
        user_id = row[user_column_name]
        item_id = row[movie_id_column]
        user_idx = user_id_to_index.get(user_id, None)

        if user_idx is None:
            continue

        # Find similar users
        similar_users_idx = np.argsort(-pcc_matrix[user_idx])[:num_similar_users + 1]
        similar_users_idx = similar_users_idx[similar_users_idx != user_idx][:num_similar_users]

        # Collect historical ratings from similar users
        similar_users_ratings = ""
        for idx in similar_users_idx:
            similar_user_id = data[user_column_name].unique()[idx]
            similar_user_data = data[data[user_column_name] == similar_user_id]
            historical_ratings = similar_user_data.sample(n=num_ratings_per_user, random_state=seed)
            for _, sim_row in historical_ratings.iterrows():
                rating_info = f"* Title: {sim_row[movie_column_name]}, Rating: {sim_row[rating_column_name]} stars"
                similar_users_ratings += rating_info + "\n"
        
        # Retrieve the main user's historical ratings
        main_user_data = data[data[user_column_name] == user_id]
        main_user_ratings = main_user_data.sample(n=num_main_user_ratings, random_state=seed)
        main_user_ratings_str = '\n'.join([
            f"* Title: {row[movie_column_name]}, Rating: {row[rating_column_name]} stars"
            for _, row in main_user_ratings.iterrows()
        ])

        # Construct prompt for API call
        combined_text = f"Title: {row[movie_column_name]}"
        prompt = f"Main User Ratings:\n{main_user_ratings_str}\n\nSimilar Users' Ratings:\n{similar_users_ratings}\n\nPredict rating for '{combined_text}':"

        predicted_rating = predict_rating_combined_ChatCompletion(
            combined_text, 
            approach="CF", 
            similar_users_ratings=similar_users_ratings,
            rating_history=main_user_ratings_str
        )

        # Update original dataframe
        original_data.loc[(original_data[user_column_name] == user_id) & (original_data[movie_id_column] == item_id), 'predicted_rating'] = predicted_rating

    original_data.to_csv(original_data_path, index=False)
    print(f"Updated predictions saved to {original_data_path}")

    return original_data


In [ ]:
# def rerun_and_update_failed_predictions(original_data_path, pcc_matrix, 
#                                         failed_indices_df, 
#                                         data, 
#                                         user_column_name='UserID', 
#                                         movie_column_name='Title', 
#                                         movie_id_column='MovieID',
#                                         rating_column_name='Rating', 
#                                         num_ratings_per_user=1, 
#                                         num_similar_users=4,
#                                         num_main_user_ratings=4,
#                                         seed=RANDOM_STATE):
#     """
#     Reruns predictions for rows with failed indices and updates the original DataFrame.
#     """
#     original_data = pd.read_csv(original_data_path)
#     user_id_to_index = {user_id: idx for idx, user_id in enumerate(data[user_column_name].unique())}

#     random.seed(seed)

#     for _, row in failed_indices_df.iterrows():
#         user_id = row[user_column_name]
#         item_id = row[movie_id_column]
#         user_idx = user_id_to_index.get(user_id, None)

#         if user_idx is None:
#             continue

#         # Find similar users
#         similar_users_idx = np.argsort(-pcc_matrix[user_idx])[:num_similar_users + 1]
#         similar_users_idx = similar_users_idx[similar_users_idx != user_idx][:num_similar_users]

#         # Collect historical ratings from similar users
#         similar_users_ratings = ""
#         for idx in similar_users_idx:
#             similar_user_id = data[user_column_name].unique()[idx]
#             similar_user_data = data[data[user_column_name] == similar_user_id]
#             historical_ratings = similar_user_data.sample(n=num_ratings_per_user, random_state=seed)
#             for _, sim_row in historical_ratings.iterrows():
#                 rating_info = f"* Title: {sim_row[movie_column_name]}, Rating: {sim_row[rating_column_name]} stars"
#                 similar_users_ratings += rating_info + "\n"
        
#         # Retrieve the main user's historical ratings from the original data
#         main_user_data = data[data[user_column_name] == user_id]
#         if len(main_user_data) >= num_main_user_ratings:
#             main_user_ratings = main_user_data.sample(n=num_main_user_ratings, random_state=seed)
#         else:
#             main_user_ratings = main_user_data

#         main_user_ratings_str = '\n'.join([
#             f"* Title: {user_row[movie_column_name]}, Rating: {user_row[rating_column_name]} stars"
#             for _, user_row in main_user_ratings.iterrows()
#         ])

#         # Construct prompt for API call
#         combined_text = f"Title: {row[movie_column_name]}"
#         prompt = f"Main User Ratings:\n{main_user_ratings_str}\n\nSimilar Users' Ratings:\n{similar_users_ratings}\n\nPredict rating for '{combined_text}':"

#         # Placeholder for the rating prediction function
#         # You need to define the `predict_rating_combined_ChatCompletion` function
#         # predicted_rating = predict_rating_combined_ChatCompletion(...)

#         # Update original dataframe
#         original_data.loc[(original_data[user_column_name] == user_id) & (original_data[movie_id_column] == item_id), 'predicted_rating'] = predicted_rating

#     original_data.to_csv(original_data_path, index=False)
#     print(f"Updated predictions saved to {original_data_path}")

#     return original_data



In [ ]:
# get column names of non_float_ratings
non_float_ratings_cols = non_float_ratings.columns.tolist()
print(f"Columns of non_float_ratings: {non_float_ratings_cols}")

# columns names of data
data_cols = data.columns.tolist()
print(f"Columns of data: {data_cols}")

Columns of non_float_ratings: ['user_id', 'item_id', 'title', 'actual_rating', 'predicted_rating', 'is_rating_float']
Columns of data: ['UserID', 'MovieID', 'Rating', 'Timestamp', 'Gender', 'Age', 'Occupation', 'Zip-code', 'Title', 'Genres']


In [ ]:
%%time 

updated_data = rerun_and_update_failed_predictions(
    original_data_path=CF_output_path,
    pcc_matrix=pcc_matrix, 
    failed_indices_df=non_float_ratings, 
    data=data,
    user_column_name='user_id',
    movie_column_name='title',
    movie_id_column='item_id',
    rating_column_name='actual_rating',
    seed=RANDOM_STATE
)


Constructed Prompt for CF approach:

The prompt:
**********
How will user rate this Title: Lethal Weapon 2 (1989)? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text.

Here is user rating history:
* Title: Lethal Weapon 2 (1989), Rating: 2 stars

Here are the rating history from users who are similar to this user:
* Title: Waterboy, The (1998), Rating: 3 stars
* Title: Jurassic Park (1993), Rating: 4 stars
* Title: Armageddon (1998), Rating: 4 stars
* Title: Star Trek: The Wrath of Khan (1982), Rating: 5 stars


Based on the above information, please predict user's rating for the product: (1 being lowest and 5 being highest, The output should be like: (x stars, xx%), do not explain the reason.)
**********



System Fingerprint: fp_772e8125bb

API call response: "2 stars, 20%"
Extracted rating: 2.0
Constructed Prompt for CF approach:

The prompt:
**********
How will user rate this Title: 12 Angry Men (1957)? (

In [ ]:
import pandas as pd

# Read the data
data = pd.read_csv(CF_output_path)

# Display the original data types
print("Original Data Types:")
print(data.dtypes)
print("\n")

# display data dimensions
print(f"Data dimensions: {data.shape}")

# Attempt to convert ratings to float and add a flag for conversion failure
data['is_rating_float'] = pd.to_numeric(data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = data[data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# rerun indices for non-float ratings
rerun_indices = non_float_ratings.index.tolist()
print(f"Rerun indices: {rerun_indices}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


Original Data Types:
user_id               int64
item_id               int64
title                object
actual_rating         int64
predicted_rating    float64
dtype: object


Data dimensions: (6040, 5)
Total number of rows with non-float ratings: 0
Rerun indices: []
Rows with non-float ratings:


,user_id,item_id,title,actual_rating,predicted_rating,is_rating_float


In [ ]:
# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_output_path,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)

RMSE: 0.4336 (95% CI: (0.4193, 0.4483)) ± 0.0001
MAE: 0.1788 (95% CI: (0.1689, 0.1889)) ± 0.0001

First few actual vs predicted ratings:
Actual: 3, Predicted: 3.0000
Actual: 2, Predicted: 2.0000
Actual: 3, Predicted: 3.0000
Actual: 2, Predicted: 3.0000
Actual: 5, Predicted: 5.0000
